# 4.Data Subscription and Visualisation

In [1]:
from IPython.display import clear_output
import json
import threading
import time
from urllib.parse import quote_plus

import pandas as pd
from sqlalchemy import create_engine, text
import paho.mqtt.client as mqtt
import folium
from ipywidgets import interact, SelectMultiple
from IPython.display import display

In [2]:
# =========================
# COMP5339 A2 - Task 4
# Realtime MQTT Subscriber + Region & Fuel Filters + Folium Map
# =========================
BROKER_HOST = "test.mosquitto.org"
BROKER_PORT = 1883
MQTT_TOPIC  = "nem/yjia0057/power_emissions"
MAP_HTML_PATH = "dashboard_map.html"
MAP_REFRESH_INTERVAL = 15.0
LOCAL_TZ = "Australia/Sydney"

# ===================== 后台预过滤条件 =====================
ACTIVE_REGION_FILTERS = []  # 例如 ["NSW1", "QLD1"]
ACTIVE_FUEL_FILTERS   = []  # 例如 ["wind", "solar_utility"]
# =========================================================

# ----------------------------------------------------------------------
# 2. 全局最新状态：latest_by_facility
# ----------------------------------------------------------------------

latest_lock = threading.Lock()
latest_by_facility: dict[str, dict] = {}


# ----------------------------------------------------------------------
# 3. MQTT 回调：合并 fuel 信息 + 存入 latest_by_facility
# ----------------------------------------------------------------------

def _to_local_sydney(ts_val):
    ts = pd.to_datetime(ts_val, errors="coerce")
    if pd.isna(ts):
        return None
    if getattr(ts, "tz", None) is None:
        try:
            return ts.tz_localize(LOCAL_TZ, ambiguous="infer", nonexistent="shift_forward")
        except Exception:
            return ts.tz_localize(LOCAL_TZ)
    else:
        return ts.tz_convert(LOCAL_TZ)


def merge_with_facility_meta(msg: dict) -> dict:
    """
    将 MQTT 消息融合 facility 的 fuel 信息，生成统一 record.
    """
    fac_code = msg.get("facility_code")
    facility_name = str(msg.get("facility_name", "")).strip()
    region = msg.get("region")
    lat = msg.get("lat")
    lon = msg.get("lon")
    fueltech = msg.get("fueltech")  

    ts_raw = msg.get("timestamp")
    try:
        ts_loc = _to_local_sydney(ts_raw)
        ts_iso_loc = ts_loc.isoformat() if ts_loc is not None else ts_raw
    except Exception:
        ts_iso_loc = ts_raw

    record = {
        "timestamp": ts_iso_loc,
        "facility_code": fac_code,
        "facility_name": facility_name or fac_code,
        "nem_region": region,
        "lat": lat,
        "lon": lon,
        "power_mw": msg.get("power_mw"),
        "co2_t": msg.get("co2_t"),
        "price": msg.get("price"),
        "demand": msg.get("demand"),
        "seq": msg.get("seq"),
        "fueltech": msg.get("fueltech")
    }
    return record


def on_connect(client, userdata, flags, reason_code, properties=None):
    print("[MQTT] Connected:", reason_code)
    client.subscribe(MQTT_TOPIC, qos=1)
    print(f"[MQTT] Subscribed to topic: {MQTT_TOPIC}")


def on_message(client, userdata, msg):
    global latest_by_facility
    try:
        payload = msg.payload.decode("utf-8")
        data = json.loads(payload)
    except Exception as e:
        print("[MQTT] Failed to decode message:", e)
        return

    record = merge_with_facility_meta(data)
    fac_code = record.get("facility_code")

    with latest_lock:
        latest_by_facility[fac_code] = record

    # 调试输出前几条
    if record.get("seq") in (1, 2, 3, 4, 5):
        print("[MQTT] sample record:", record)


def start_mqtt_subscriber():
    client = mqtt.Client()
    client.on_connect = on_connect
    client.on_message = on_message

    client.connect(BROKER_HOST, BROKER_PORT, keepalive=60)

    t = threading.Thread(target=client.loop_forever, daemon=True)
    t.start()
    print("[MQTT] Subscriber loop started (background).")
    return client


# ----------------------------------------------------------------------
# 4. 地图生成：Region + Fuel 双图层
# ----------------------------------------------------------------------

def make_map_from_latest(latest_records: list[dict]) -> folium.Map:
    """
    使用 latest_by_facility 里的记录生成 folium 地图。
    图层设计：
      - Region 图层：Region: NSW1 / Region: QLD1 / ...
      - Fuel 图层：Fuel: wind / Fuel: solar_utility / ...
      - 每个点会被加到对应的 region 图层 + fuel 图层
      - 所有图层在 HTML 中默认 show=False（进入页面时全部不勾选）
    """
    # 0. 没数据的情况
    if not latest_records:
        center_lat, center_lon = -25.0, 135.0
        fmap = folium.Map(location=[center_lat, center_lon],
                          zoom_start=5, tiles="cartodbpositron")
        folium.Marker(
            location=[center_lat, center_lon],
            popup="No MQTT data received yet...",
            icon=folium.Icon(color="gray", icon="info-sign")
        ).add_to(fmap)
        return fmap

    df = pd.DataFrame(latest_records)

    # 1. 后台预过滤
    cond = pd.Series(True, index=df.index)
    if ACTIVE_REGION_FILTERS:
        cond &= df["nem_region"].isin(ACTIVE_REGION_FILTERS)
    if ACTIVE_FUEL_FILTERS:
        cond &= df["fueltech"].isin(ACTIVE_FUEL_FILTERS)

    df = df[cond].copy()

    if df.empty:
        center_lat, center_lon = -25.0, 135.0
        fmap = folium.Map(location=[center_lat, center_lon],
                          zoom_start=5, tiles="cartodbpositron")
        folium.Marker(
            location=[center_lat, center_lon],
            popup="No facilities match the selected backend filters.",
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(fmap)
        return fmap

    # 2. 地图中心
    lat_valid = df["lat"].dropna()
    lon_valid = df["lon"].dropna()
    if len(lat_valid) and len(lon_valid):
        center_lat = lat_valid.mean()
        center_lon = lon_valid.mean()
    else:
        center_lat, center_lon = -25.0, 135.0

    fmap = folium.Map(location=[center_lat, center_lon],
                      zoom_start=5, tiles="cartodbpositron")

    # 3. Header 信息（时间 + 价格 + 需求 + 当前后台过滤）
    ts_series = pd.to_datetime(df["timestamp"], errors="coerce")
    latest_ts = ts_series.max()
    if pd.notna(latest_ts):
        df_latest = df[ts_series == latest_ts]
    else:
        df_latest = df

    avg_price = df_latest["price"].dropna().mean()
    sum_demand = df_latest["demand"].dropna().sum()

    region_text = ", ".join(ACTIVE_REGION_FILTERS) if ACTIVE_REGION_FILTERS else "ALL"
    fuel_text = ", ".join(ACTIVE_FUEL_FILTERS) if ACTIVE_FUEL_FILTERS else "ALL"

    header_html = f"""
    <div style="position: fixed; 
                top: 10px; left: 50px; z-index: 9999; 
                background-color: white;
                padding: 8px 12px; 
                border: 1px solid #999;
                border-radius: 4px;
                box-shadow: 1px 1px 4px rgba(0,0,0,0.3);">
        <b>NEM Dashboard</b><br/>
        Latest timestamp: {latest_ts}<br/>
        Avg Price: {avg_price:.2f} $/MWh (approx.)<br/>
        Total Demand: {sum_demand:.2f} MW (approx.)<br/>
        Backend Region filter: {region_text}<br/>
        Backend Fuel filter: {fuel_text}
    </div>
    """
    fmap.get_root().html.add_child(folium.Element(header_html))

    # 4. fueltech → 颜色映射（按 fuel 上色）
    fuel_colors = {
        "wind": "green",
        "solar_utility": "orange",
        "solar_rooftop": "lightred",
        "battery": "purple",
        "battery_charging": "gray",
        "hydro": "blue",
        "coal_black": "black",
        "coal_brown": "darkred",
        "gas_ccgt": "red",
        "gas_ocgt": "darkblue",
        "gas_recip": "pink",
        "gas_steam": "lightblue",
        "gas_wcmg": "cadetblue",
        "bioenergy_biogas": "darkgreen",
        "bioenergy_biomass": "lightgreen",
        "pumps": "beige",
    }

    # 5. Region 图层（默认不勾选）
    region_groups = {}
    for reg in sorted(df["nem_region"].dropna().unique()):
        fg = folium.FeatureGroup(name=f"Region: {reg}", show=False)
        fg.add_to(fmap)
        region_groups[reg] = fg

    # 6. Fuel 图层
    fuel_groups = {}
    if "fueltech" in df.columns:
        fuels = sorted(df["fueltech"].dropna().unique())
        for fuel in fuels:
            fg = folium.FeatureGroup(name=f"Fuel: {fuel}", show=False)
            fg.add_to(fmap)
            fuel_groups[fuel] = fg

    # 7. 往两个图层里加点
    for _, row in df.iterrows():
        lat = row.get("lat")
        lon = row.get("lon")
        if pd.isna(lat) or pd.isna(lon):
            continue

        ps_name = row.get("facility_name")
        fac_code = row.get("facility_code")
        reg = row.get("nem_region")
        fuel = row.get("fueltech")
        power_mw = row.get("power_mw")
        co2_t = row.get("co2_t")
        price = row.get("price")
        demand = row.get("demand")
        ts = row.get("timestamp")

        popup_html = f"""
        <b>{ps_name}</b><br/>
        Facility code: {fac_code}<br/>
        NEM Region: {reg}<br/>
        Fueltech: {fuel}<br/>
        <br/>
        Timestamp: {ts}<br/>
        Power: {power_mw} MW<br/>
        Emissions: {co2_t} tCO₂<br/>
        Price: {price} $/MWh<br/>
        Demand: {demand} MW
        """

        color = fuel_colors.get(str(fuel), "gray")

        # 这个点对应的 region / fuel 图层
        targets = []

        reg_group = region_groups.get(reg)
        if reg_group is not None:
            targets.append(reg_group)

        fuel_group = fuel_groups.get(fuel)
        if fuel_group is not None:
            targets.append(fuel_group)

        if not targets:
            targets = [fmap]

        for g in targets:
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(popup_html, max_width=300, min_width=200),
                icon=folium.Icon(color=color, icon="bolt", prefix="fa")
            ).add_to(g)

    # 8. LayerControl
    folium.LayerControl(collapsed=False).add_to(fmap)
    return fmap

def map_refresher_loop():
    print(f"[MAP] Refresher started, interval = {MAP_REFRESH_INTERVAL}s")
    while True:
        time.sleep(MAP_REFRESH_INTERVAL)
        with latest_lock:
            records = list(latest_by_facility.values())
        try:
            fmap = make_map_from_latest(records)
            fmap.save(MAP_HTML_PATH)
            #clear_output(wait=True)
            print(f"[MAP] Updated {len(records)} facilities -> {MAP_HTML_PATH}")
        except Exception as e:
            print("[MAP] Failed to update map:", e)

# ----------------------------------------------------------------------
# 5. main
# ----------------------------------------------------------------------

def main(run_duration=300):
    print("=== COMP5339 Assignment 2 - Task 4 Dashboard ===")
    print(f"[INFO] Region filters (backend): {ACTIVE_REGION_FILTERS or 'ALL'}")
    print(f"[INFO] Fuel filters   (backend): {ACTIVE_FUEL_FILTERS or 'ALL'}")

    start_mqtt_subscriber()

    t_map = threading.Thread(target=map_refresher_loop, daemon=True)
    t_map.start()

    print(f"[INFO] Running for {run_duration} seconds. "
          f"Open '{MAP_HTML_PATH}' in your browser and refresh periodically.")
    try:
        time.sleep(run_duration)
    except KeyboardInterrupt:
        print("\n[MAIN] Stopped by user.")
    print("[MAIN] Dashboard finished.")

In [3]:
main(run_duration=100)  # 运行n分钟后自动退出

=== COMP5339 Assignment 2 - Task 4 Dashboard ===
[INFO] Region filters (backend): ALL
[INFO] Fuel filters   (backend): ALL


C:\Users\86166\AppData\Local\Temp\ipykernel_35052\526362164.py:104: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


[MQTT] Subscriber loop started (background).
[MAP] Refresher started, interval = 15.0s
[INFO] Running for 100 seconds. Open 'dashboard_map.html' in your browser and refresh periodically.
[MQTT] Connected: 0
[MQTT] Subscribed to topic: nem/yjia0057/power_emissions
[MAP] Updated 72 facilities -> dashboard_map.html
[MAP] Updated 147 facilities -> dashboard_map.html
[MAP] Updated 222 facilities -> dashboard_map.html
[MAP] Updated 240 facilities -> dashboard_map.html
[MAP] Updated 240 facilities -> dashboard_map.html
[MAP] Updated 240 facilities -> dashboard_map.html
[MAIN] Dashboard finished.
